## DABs from the Ground Up

### 1. Create a `databricks.yml` file
```yaml
bundle:
  name: thhart_dab_sql

# specify variables but do not assign values
variables:
  warehouse_id: 
    description: The warehouse to use
  catalog: 
    description: The catalog to use
  schema:
    description: The schema to use

# specify variables values in targets
targets:
  
  # where do we want to develop
  thhart_target_dev:
    mode: development
    default: true
    workspace: 
      host: https://adb-984752964297111.11.azuredatabricks.net
    variables:
      warehouse_id: 148ccb90800933a1
      catalog: thhart
      schema: dab_sql_dev
  
  # where do we want to run in production
  thhart_target_prod:
    mode: production
    workspace: 
      host: https://adb-984752964297111.11.azuredatabricks.net
    variables:
      warehouse_id: 148ccb90800933a1
      catalog: thhart
      schema: dab_sql_prod
```

In [0]:
%sql 
create schema thhart.dab_sql_dev;
create schema thhart.dab_sql_prod;

### 2. Explore Data

In [0]:
%sql list '/databricks-datasets/retail-org/sales_orders'

In [0]:
%sql
select * 
from read_files(
  '/databricks-datasets/retail-org/sales_orders'
  , format => 'json'
  , header => true
)
limit 5

### 3. Create Logic

In [0]:
%sql
select 
  customer_id
  , order_number
  , from_unixtime(order_datetime::bigint) as order_datetime
from read_files(
  '/databricks-datasets/retail-org/sales_orders'
  , format => 'json'
  , header => true
)
limit 5

In [0]:
%sql
with cte as (
-- orders
select 
  customer_id
  , order_number
  , from_unixtime(order_datetime::bigint) as order_datetime
from read_files(
  '/databricks-datasets/retail-org/sales_orders'
  , format => 'json'
  , header => true
))

-- orders daily
select 
  order_datetime::date as order_date
  , count(*) as n_orders
from cte
group by order_datetime::date
limit 5

### 4. Link SQL Together
Two way to achieve this
* Use interface to create a job and copy yaml
* Write yaml 
* Jobs
```yaml
resources:
  jobs:
    thhart_dab_sql_job_yml:
      name: thhart_dab_sql_job_yml

      trigger:
          interval: 1
          unit: DAYS
      
      email_notifications:
        on_failure:
          - thomas.hart@databricks.com

      parameters:
        - name: thhart_catalog
          default: ${var.thhart_catalog}
        - name: thhart_schema
          default: ${var.thhart_schema}
        - name: bundle_target
          default: ${bundle.target}

      tasks:
        - task_key: orders_raw
          sql_task:
            warehouse_id: ${var.warehouse_id}
            file:
              path: ./orders_raw.sql

        - task_key: orders_daily
          depends_on:
            - task_key: orders_raw
          sql_task:
            warehouse_id: ${var.warehouse_id}
            file:
              path: ./orders_daily.sql
```

### Deploy